In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # to read records completely
import pickle
import numpy as np
import itertools
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

## Previous data prep necessary for c) and d)

In [2]:
df = pd.read_pickle('data_clean.pkl')

In [3]:
models = pd.read_csv('models.csv', header=None, names = ['brand', 'model'])
models['brand'] = models['brand'].str.replace(r'[^\w\s]+', '', regex=True) # clean some punctuation errors (nissan.)
models = models[~models.brand.isin(['car', 'sedan', 'problem'])] # remove rows that are not brands

In [4]:
df.Message.str.contains('said').sum()

370

In [5]:
df.Message_words = df.Message_words.apply(lambda x: [i.lower() for i in x])

### Replace models with brands

In [6]:
messages = df.Message_words.to_list()

In [7]:
# numpy.where to do the replacement. Replaces only first appearance of brand.
# ~ 5-10 minutes of execute
messages2 = []
t = 0
for m in messages:
    for i,row in models.iterrows():
        m = np.where(m == row['model'], row['brand'], m)
        
    t+=1
    
    #print(t)
        
    messages2.append(m)
        
df['Message_words_v2'] = messages2

In [8]:
# let's find the occurences of changes
m1 = df.Message_words.to_list()
m2 = df.Message_words_v2.to_list()

c = 0
W = 0
for i in range(len(m2)):
    for j in range(len(m2[i])):
        if m1[i][j] != m2[i][j]: #word by word comparison
            c +=1
        W += 1 # if we want to count all words

In [9]:
print(f'{c} changes of models')

6368 changes of models


In [10]:
df[['Message_words', 'Message_words_v2']].head(10)

,Message_words,Message_words_v2
0,"[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]"
1,"[im, buying, new, car, end, year, first, choice, nissan, maxima, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]"
2,"[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]"
3,"[looking, 1, 4, choices, replace, 99gs400, ive, read, everything, forums, seen, much, conflicting, info4, optionslexus, ls430, customluxury, packageinfiniti, q45, premium, pckgaudi, a6, 42, loaded, audi, a8, stripped, loadedchicago, wintershad, blizzaks, gsworked, fine, 95q, pig, snow, im, sure, 2002q, world, away, ive, heard, audi, qc, problems, like, maintenance, included, im, happy, lexus, driver, q, appears, offer, lease, economy, would, think, could, command, attractive, package, go, wrong, 4, missing, something]","[looking, 1, 4, choices, replace, 99gs400, ive, read, everything, forums, seen, much, conflicting, info4, optionslexus, ls430, customluxury, packageinfiniti, q45, premium, pckgaudi, audi, 42, loaded, audi, audi, stripped, loadedchicago, wintershad, blizzaks, gsworked, fine, 95q, pig, snow, im, sure, 2002q, world, away, ive, heard, audi, qc, problems, like, maintenance, included, im, happy, lexus, driver, q, appears, offer, lease, economy, would, think, could, command, attractive, package, go, wrong, 4, missing, something]"
4,"[generally, 34, year, old, car, range, good, one, basically, sound, possible, get, one, still, warrenty, make, sure, trusted, maxima, mechanic, goes, thoroughly, v6, one, best, rated, 10, years, personally, amazed, great, strides, american, quality, recently, olds, may, bad, idea]","[generally, 34, year, old, car, range, good, one, basically, sound, possible, get, one, still, warrenty, make, sure, trusted, nissan, mechanic, goes, thoroughly, v6, one, best, rated, 10, years, personally, amazed, great, strides, american, quality, recently, olds, may, bad, idea]"
5,"[maxima, intrigue, excellent, choices, midsize, sedans]","[nissan, intrigue, excellent, choices, midsize, sedans]"
6,"[kind, budget, would, go, a6, 42, best, combination, sport, luxury, amongst, 4, choices]","[kind, budget, would, go, audi, 42, best, combination, sport, luxury, amongst, 4, choices]"
7,"[actually, agree, a6, looked, specs, edmunds, a6, a8, 4000, lbs, considering, a8, gets, bump, hp, a6, dont, think, performance, difference, issue, considering, fact, a6, would, possibly, small, based, rest, list, say, go, a8, think, youll, find, awd, best, addition, car, could, ever, hoped]","[actually, agree, aud

In [11]:
#Now we have cases where the text says "nissan", "nissan" because we replaced the model. Let's clean that
#Removing duplicates in lists while keeping order
#import itertools
def remove_consecutive_duplicate (text):
    return np.array([i for i, j in itertools.groupby(text)])

df["Message_words_v3"] = df["Message_words_v2"].apply(remove_consecutive_duplicate)

## Link attributes to models

In [12]:
# first lets count brand mentions in each record

In [13]:

#Get a unique list of brands
brands_list = models.brand.drop_duplicates().to_list()

brands_list.extend(['lexus', 'ferrari', 'merzedesbenz', 'tesla','gm', 'peugeot', 'jeep', 'bentley', 'fiat'])

In [14]:
messages = df.Message_words_v3.to_list()


brands_in_message = []
for m in messages:
    
    brands_mentioned = []
    for brand in brands_list:
        if len(np.where(m == brand)[0]) >0:
            brands_mentioned.append(brand)
            
    brands_in_message.append(brands_mentioned)
        
    

C:\Users\louis\AppData\Local\Temp/ipykernel_21972/1539195482.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if len(np.where(m == brand)[0]) >0:


In [15]:
df['Brands_in_message'] = brands_in_message
df['Qty_brands'] = df.Brands_in_message.apply(lambda x: len(x))
df['Qty_brands'].value_counts()

# 1291 messages don't mention any brand (remove them from analysis?)
# Most messages discuss a single brand
# Also common to compare 2 brands

0     1244
1     1175
2      774
3      464
4      235
5      120
6       74
7       31
8       17
10       7
11       3
12       3
9        2
16       1
13       1
Name: Qty_brands, dtype: int64

In [16]:
df.loc[df.Qty_brands == 0, 'Message_words_v3'].head(10) # to review records with no brand mentions
# most of them are for seeking advice

0                                                                                       [need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]
2     [buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]
8                                                                                                                                                                                                                                                                                      [id, pass, intrigue, one, reaso

In [70]:
# First approach: link all words found between brand mention and next brand mention.

# Except for first brand which also receives words since beginning of message

m = df.Message_words_v3[9] #record 9 is a good exaple

def get_attr_in_the_right(m, brand_list):

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]


    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

    brand_list = list(brand_dic.values()) + [len(m)]

    review={}
    for i, tup in enumerate(brand_dic):
#         print(brand_dic[tup])
#         print(tup)
        
        if i == 0:
            
            review[tup] = m[0:brand_list[1]]
        else:
            review[tup] = m[brand_list[i]:brand_list[i+1]]
            
    return review



get_attr_in_the_right(m, brands_list) #was brand_list, ASK CARLOS    

{'honda': array(['would', 'recommend', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
        '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [71]:
# now we run it for all records

brand_reviews_1st_approach = []

for m in df.Message_words_v3:

    brand_reviews_1st_approach.append(get_attr_in_the_right(m, brands_list))

# generates a list (1 entry per row) of dictionaries {brand: part of text corresponding}

C:\Users\louis\AppData\Local\Temp/ipykernel_21972/3694316805.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ix = np.where(m == brand)[0]


In [72]:
# 2nd approach: 5 words to each side 

# First approach: link all words found between brand mention and next brand mention.

# Except for first brand which also receives words since beginning of message

m = df.Message_words_v3[9] #record 9 is a good exaple

def get_attr_n_per_side(m, n, brand_list):

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]


    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))
    
    #print(len(brand_dic))

    brand_list = list(brand_dic.values()) + [len(m)]
    
    #print(brand_list)

    review={}
    for i, tup in enumerate(brand_dic):
#         print(brand_dic[tup])
#         print(tup)

        left_loc = (brand_list[i] - n)
        left_loc = max(0, left_loc) # to avoid out of range indexing
        
        right_loc = (brand_list[i] + n)
        right_loc = min(len(m), right_loc) # to avoid OOF indexing
        
        review[tup] = m[left_loc:right_loc]
            
    return review



get_attr_n_per_side(m, 5, brands_list)        

{'honda': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new'],
       dtype='<U12'),
 'toyota': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new',
        'nissan'], dtype='<U12'),
 'nissan': array(['honda', 'toyota', 'choices', 'sedan', 'new', 'nissan', 'looks',
        'impressive', 'would', 'wait'], dtype='<U12'),
 'bmw': array(['wait', 'reliability', '2002', 'proventhe', 'new', 'bmw', '525',
        'another', 'nice', 'car'], dtype='<U12')}

In [73]:
m = df.Message_words_v3[9] 

m

array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new',
       'nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
       '2002', 'proventhe', 'new', 'bmw', '525', 'another', 'nice', 'car',
       'rated', 'highly', 'many', 'auto', 'publications'], dtype='<U12')

In [74]:
get_attr_n_per_side(m, 4, brands_list)

{'honda': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan'],
       dtype='<U12'),
 'toyota': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new'],
       dtype='<U12'),
 'nissan': array(['toyota', 'choices', 'sedan', 'new', 'nissan', 'looks',
        'impressive', 'would'], dtype='<U12'),
 'bmw': array(['reliability', '2002', 'proventhe', 'new', 'bmw', '525', 'another',
        'nice'], dtype='<U12')}

In [75]:
get_attr_in_the_right(m, brands_list)        

{'honda': array(['would', 'recommend', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
        '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [76]:
# now we run it for all records

brand_reviews_2nd_approach = []

for m in df.Message_words_v3:

    brand_reviews_2nd_approach.append(get_attr_n_per_side(m,4, brands_list))

# generates a list (1 entry per row) of dictionaries {brand: part of text corresponding}

C:\Users\louis\AppData\Local\Temp/ipykernel_21972/1876655006.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ix = np.where(m == brand)[0]


In [77]:
len(brand_reviews_2nd_approach) # each record becomes a single dictionary with all its brand mentions and linked words

4151

In [78]:
# now we need to concatenate all reviews for each brand into a single key in a dictionary
review_accum_1st = {i:[] for i in brands_list}

for brand in brands_list:
    for review in brand_reviews_1st_approach:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum_1st[brand].extend(review[single_review])

In [79]:
# now we need to concatenate all reviews for each brand into a single key in a dictionary
review_accum_2nd = {i:[] for i in brands_list}

for brand in brands_list:
    for review in brand_reviews_2nd_approach:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum_2nd[brand].extend(review[single_review])

In [80]:
review_accum_1st

{'acura': ['acura',
  'come',
  'v8',
  'model',
  'acura',
  'rl',
  'might',
  'try',
  'acura',
  'tl',
  'smallest',
  'caddy',
  'rwd',
  'ctsid',
  'pick',
  'acura',
  'tl',
  'type',
  'best',
  'value',
  'edmunds',
  'editors',
  'choice',
  'well',
  'dont',
  'remember',
  'exactly',
  'going',
  'know',
  'got',
  'ton',
  'bad',
  'press',
  'sully',
  'name',
  'bit',
  'kind',
  'like',
  'isuzu',
  'trooper',
  'flipping',
  'killed',
  'acura',
  'version',
  'remember',
  'current',
  'grand',
  'cherokee',
  'sudden',
  'acceleration',
  'im',
  'looking',
  'buying',
  'new',
  'family',
  'sedan',
  'near',
  'luxury',
  'class',
  'looking',
  '1',
  'acura',
  'tl',
  '32',
  'type',
  '2',
  '02',
  'acura',
  'acura',
  'great',
  'acura',
  'well',
  'sporty',
  'nissan',
  'upgrade',
  'honda',
  'think',
  'claywater',
  'godeacs',
  'dead',
  'cars',
  'near',
  'luxury',
  'arena',
  'great',
  'bang',
  'buck',
  'cars',
  'nice',
  'engines',
  'interio

In [81]:
review_accum_2nd

{'acura': ['leftover',
  'available',
  'scramble',
  'thinking',
  'acura',
  'come',
  'v8',
  'model',
  'range',
  'new',
  'toyota',
  'told',
  'acura',
  'rl',
  'infiniti',
  'j30',
  'might',
  'try',
  'acura',
  'tl',
  'lexus',
  'es300',
  'caddy',
  'rwd',
  'ctsid',
  'pick',
  'acura',
  'tl',
  'type',
  'best',
  'isuzu',
  'trooper',
  'flipping',
  'killed',
  'acura',
  'version',
  'remember',
  'current',
  'luxury',
  'class',
  'looking',
  '1',
  'acura',
  'tl',
  '32',
  'type',
  'would',
  'go',
  'toyota',
  'xle',
  'acura',
  'nissan',
  'look',
  'sharper',
  'concerned',
  'primarily',
  'resale',
  'honda',
  'acura',
  'great',
  'toyota',
  'bad',
  'may',
  'want',
  'take',
  'look',
  'acura',
  'well',
  'sporty',
  'nissan',
  'interiors',
  'wont',
  'kill',
  'wallet',
  'acura',
  'may',
  'boring',
  'looking',
  'rough',
  'time',
  'deciding',
  'acura',
  'tls',
  'audi',
  '18t',
  'like',
  'acura',
  'higher',
  'reliability',
  'plu

In [ ]:
## from here, we are ready to start counting attributes per brand
## we also need to consider replacing some similar attributes before doing the final count 
## I will do one example with Audi but it could become a function and run in a loop to all the brands 

In [82]:
# first lets discover popular attributes regardless of brands

all_attributes = []

for i in list(review_accum.values()):
    all_attributes.extend(i)
    
words = pd.Series(all_attributes)

words.value_counts().head(40) # this shows the frequencies of each word
# the most "rustic" approach would be to literally print them in order and take note of the ones that can be considered "attributes"

NameError: name 'review_accum' is not defined

In [ ]:
words[~words.isin(brands_list)].value_counts().head(60)


In [ ]:
words_count = words.value_counts()
words_nobrands_count = words[~words.isin(brands_list)].value_counts()

### Stemming

In [ ]:
#Stemming
stemmer = PorterStemmer()

In [ ]:
def stem_list (text):
    return pd.Series([stemmer.stem(w) for w in text])

In [ ]:
stemmed_words = stem_list(words)
len(stemmed_words)

In [ ]:
stemmed_words.value_counts().head(60)

In [ ]:
stemmed_words[~stemmed_words.isin(brands_list)].value_counts().head(60)


In [ ]:
stemwords_count = stemmed_words.value_counts()
stemwords_nobrands_count = stemmed_words[~stemmed_words.isin(brands_list)].value_counts()

In [ ]:
#For words_nobrands_count remove words that appear 2x or less
words_nobrands_count = words_nobrands_count[words_nobrands_count > 2]
words_nobrands_count

In [ ]:
# This csv is used to create our attribute map
#words_nobrands_count.to_csv("attribute_count_final.csv")

### Importing attribute map

In [50]:
df_map = pd.read_excel("attribute_count_final.xlsx", sheet_name = 0)

In [51]:
df_map = df_map.drop('Unnamed: 4', axis=1)

In [58]:
df_map

,word,freq,attribute_flg,attribute_synonym
0,price,737.0,1.0,affordability
1,money,338.0,1.0,affordability
2,lease,311.0,1.0,affordability
3,value,273.0,1.0,affordability
4,cost,270.0,1.0,affordability
...,...,...,...,...
5356,nope,3.0,0.0,nope
5357,scrape,3.0,0.0,scrape
5358,850.0,3.0,0.0,850.0
5359,modelsi,3.0,0.0,modelsi


In [53]:
#Replace np.nan in df_map
df_map['attribute_flg'] = df_map['attribute_flg'].replace(np.nan, 0)
df_map.attribute_synonym.fillna(df_map.word, inplace=True)


In [64]:
df_pivot_map = df_map[df_map.attribute_flg == 1.0].groupby('attribute_synonym').freq.sum().sort_values(ascending=False)

In [91]:
df_pivot_map.head(10)

attribute_synonym
affordability     4509.0
sustainability    1902.0
size              1829.0
driveability      1535.0
engine            1433.0
visual            1202.0
reliable          1012.0
transmission       998.0
comfort            893.0
technological      867.0
Name: freq, dtype: float64

In [ ]:
# The top 5 attributes are: affordability, sustainability, size, driveability and engine

In [115]:
replace_lol = df.copy()

In [125]:
# Replace the attributes in df.Message_words_v3
def replace_attributes(m):
    for i,row in df_map.iterrows():
        m = np.where(m == row['word'], row['attribute_synonym'], m)
    return m

In [130]:
replace_attributes(df.Message_words_v3.loc[2])

C:\Users\louis\AppData\Local\Temp/ipykernel_21972/764214923.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  m = np.where(m == row['word'], row['attribute_synonym'], m)


array(['buy', '2001', '2002', 'oldsmobile', 'intrigue', 'one', 'best',
       'sedans', 'market', 'great', 'driveability', 'one', 'best',
       'engine', 'engine', 'around', '35l', 'dohc', 'producing', '215hp',
       'deals', 'great', 'year', 'last', 'intrigue', 'warrany',
       'extended', '5', 'years', 'car', 'recommended', 'magazines',
       'including', 'consumer', 'reports', 'unfortunately', 'looked',
       'consumers', 'sleeper', 'sleepers', 'market', 'opinion'],
      dtype='<U158')

In [134]:
# Replaces attributes
# Takes ~20-30 mins to run
df["Message_words_v3"] = df["Message_words_v3"].apply(replace_attributes)

C:\Users\louis\AppData\Local\Temp/ipykernel_21972/764214923.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  m = np.where(m == row['word'], row['attribute_synonym'], m)


# Calculating lift scores

In [139]:
def calculate_lift(brand_name, attribute_name):
    #Get total number of reviews
    review_count = len(df)
    print(f"There are {review_count} reviews")

    #Count number of times brand appears across reviews
    brand_count = 0
    for i in  df.Brands_in_message:
        if brand_name in i:
            brand_count += 1
    print(f"{brand_name} appears {brand_count} times")
    
    #Get number of times attribute appears across reviews
    #This is already counted in df_pivot_map
    attribute_count = df_pivot_map[attribute_name] 
    print(f"{attribute_name} appears {attribute_count} times")

    #Count number of times attribute and brand appear together
    attr_brand_count = 0
    for review in df.Message_words_v3:
        if brand_name in review and attribute_name in review:
            attr_brand_count += 1
    print(f"{attribute_name} and {brand_name} appears together {attr_brand_count} times")
    
    #Calculate lift
    lift = review_count * (attr_brand_count/(brand_count * attribute_count))
    
    return lift

In [140]:
calculate_lift('nissan', 'affordability')

There are 4151 reviews
nissan appears 525 times
affordability appears 4509.0 times
affordability and nissan appears together 323 times


0.5663901825977674

In [141]:
top_attributes = ['affordability', 'sustainability', 'size', 'driveability', 'engine']
top_brands = ['honda','toyota','nissan','volkswagen','chevrolet']

In [143]:
df_lift = pd.DataFrame(index=top_brands, columns = top_attributes)
df_lift

,affordability,sustainability,size,driveability,engine
honda,NaN,NaN,NaN,NaN,NaN
toyota,NaN,NaN,NaN,NaN,NaN
nissan,NaN,NaN,NaN,NaN,NaN
volkswagen,NaN,NaN,NaN,NaN,NaN
chevrolet,NaN,NaN,NaN,NaN,NaN


In [144]:
for brand in top_brands:
    for attribute in top_attributes:
        df_lift.loc[brand, attribute] = calculate_lift(brand, attribute)

There are 4151 reviews
honda appears 924 times
affordability appears 4509.0 times
affordability and honda appears together 562 times
There are 4151 reviews
honda appears 924 times
sustainability appears 1902.0 times
sustainability and honda appears together 354 times
There are 4151 reviews
honda appears 924 times
size appears 1829.0 times
size and honda appears together 344 times
There are 4151 reviews
honda appears 924 times
driveability appears 1535.0 times
driveability and honda appears together 250 times
There are 4151 reviews
honda appears 924 times
engine appears 1433.0 times
engine and honda appears together 298 times
There are 4151 reviews
toyota appears 769 times
affordability appears 4509.0 times
affordability and toyota appears together 467 times
There are 4151 reviews
toyota appears 769 times
sustainability appears 1902.0 times
sustainability and toyota appears together 314 times
There are 4151 reviews
toyota appears 769 times
size appears 1829.0 times
size and toyota appea

In [145]:
df_lift

,affordability,sustainability,size,driveability,engine
honda,0.559934,0.836129,0.844939,0.731665,0.934224
toyota,0.559066,0.891139,0.894243,0.773643,0.794809
nissan,0.56639,0.756579,0.929433,0.721129,0.90488
volkswagen,0.520016,0.666529,0.828078,0.760109,0.822042
chevrolet,0.549084,0.945508,1.124671,0.978981,1.151812


In [ ]:
# once we have the list of the 5 most popular attributes, my guess is we have to calculate the uplift scores of said attribute
# against the 5 top brands found in letter A). Then the uplift should tell us which brand is more related to which attribute

In [ ]:
# let's say "reliability" is one of the popular attributes 
# and the top 5 brands are: honda, nissan, audi, bwm, toyota
# let's get the inputs necessary to calculate uplift of reliability in nissan

In [ ]:
n=0
for i in  df.Brands_in_message:
    if 'nissan' in i:
        n += 1
n
#nissan appears in 525 rows 

In [ ]:
df_nissan = pd.DataFrame(review_accum['nissan'], columns = ['word'])
(df_nissan.word == 'reliability').sum() # 73 mentions of "relibaility" in honda

# and 28 times the term "reliability" was mentioned alongside a nissan

In [ ]:
# how about honda?

n=0
for i in  df.Brands_in_message:
    if 'honda' in i:
        n += 1
print(n)


df_honda = pd.DataFrame(review_accum['honda'], columns = ['word'])
(df_honda.word == 'reliability').sum() # 73 mentions of "relibaility" in honda


In [ ]:
73/924 # not the exact uplift formula but a quick validation

In [ ]:
28/525 # vs nissan

In [ ]:
# honda has more prevalence as a reliable brand than nissan (pending to adjust for uplift real formula)

to consider:

right now the code counts all the appearances of an attribute, even if they come from the same record; I think that is not the exact definition of the uplift formula. I don't think it will change the numbers very much, but would be good to refine it if we have the time